# Day 2: Password Philosophy

Your input is a list of passwords and the corporate policy when that password was set.

    1-3 a: abcde
    1-3 b: cdefg
    2-9 c: ccccccccc

## Part 1

Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

How many passwords are valid according to their policies?

## Solution

Parsing the input is harder than solving the problem. Let's start by defining a type that can represent this input.

In [1]:
#!fsharp
type Input = {
    FstNumber: int
    SndNumber: int
    Letter: char
    Password: string
}

// Some helper functions to make working with values of type Input easier.
module Input =
    let create fstNumber sndNumber letter password = {
        FstNumber = fstNumber
        SndNumber = sndNumber
        Letter = letter
        Password = password
    }


The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In [1]:
#!fsharp
let policy ({ FstNumber = fstNumber; SndNumber = sndNumber; Letter = letter; Password = password }: Input): bool =
    password
    |> Seq.sumBy (fun char -> if char = letter then 1 else 0)
    |> fun sum -> sum >= fstNumber && sum <= sndNumber

// Test the policy using the example provided in the puzzle.
[
    {| Input = { FstNumber = 1; SndNumber = 3; Letter = 'a'; Password = "abcde" }; ExpectedValid = true |}
    {| Input = { FstNumber = 1; SndNumber = 3; Letter = 'b'; Password = "cdefg" }; ExpectedValid = false |}
    {| Input = { FstNumber = 2; SndNumber = 9; Letter = 'c'; Password = "ccccccccc" }; ExpectedValid = true |}
] |> Seq.map (fun test -> 
    let actual = policy test.Input
    let result = if actual = test.ExpectedValid then "OK" else "NOK"
    {| test with ActualValid = policy test.Input; Result = result |})

index,ActualValid,ExpectedValid,Input,Result
0,True,True,"{ FstNumber = 1 SndNumber = 3 Letter = 'a' Password = ""abcde"" }",OK
1,False,False,"{ FstNumber = 1 SndNumber = 3 Letter = 'b' Password = ""cdefg"" }",OK
2,True,True,"{ FstNumber = 2 SndNumber = 9 Letter = 'c' Password = ""ccccccccc"" }",OK


Now that our tests pass, let's try out our solution on the actual input. The input is provided in a txt file and contains 1000 lines.

> 9-12 q: qqqxhnhdmqqqqjz
> 12-16 z: zzzzzznwlzzjzdzf
> 4-7 s: sssgssw
> 13-14 p: pppqzpppppppfpppp
> 2-9 m: jknmmmmmmdmmmrm
> 4-5 b: bbbrb
> 2-10 c: hcpzpjclzc
> 7-11 f: ffpffffgfrff
> ...

Let's write a parser that takes this blob of text and converts it to a value of type ``Input seq``.


In [1]:
#!fsharp
open System.Text.RegularExpressions

let (|Regex|_|) pattern input =
    let m = Regex.Match(input, pattern)
    if m.Success then Some(List.tail [ for g in m.Groups -> g.Value ])
    else None

let parseLine = function
    | Regex @"([0-9]+)-([0-9]+) ([a-z]): ([a-z]+)" [ first; second; letter; password ] ->
        Some (Input.create (int first) (int second) (char letter) (string password))
    | "" -> None
    | x -> failwithf "Failed to parse line %s" x

// Test the regex parseLine function using the example provided in the puzzle.
[
    {| Input = "1-3 a: abcde"; ExpectedOutput = { FstNumber = 1; SndNumber = 3; Letter = 'a'; Password = "abcde" } |}
    {| Input = "1-3 b: cdefg"; ExpectedOutput = { FstNumber = 1; SndNumber = 3; Letter = 'b'; Password = "cdefg" } |}
    {| Input = "2-9 c: ccccccccc"; ExpectedOutput = { FstNumber = 2; SndNumber = 9; Letter = 'c'; Password = "ccccccccc" } |}
] |> Seq.map (fun test -> 
    let actualOutput = parseLine test.Input
    let result = if actualOutput = Some test.ExpectedOutput then "OK" else "NOK"
    {| test with ActualOutput = actualOutput; Result = result |})


index,ActualOutput,ExpectedOutput,Input,Result
0,"Some({ FstNumber = 1 SndNumber = 3 Letter = 'a' Password = ""abcde"" })","{ FstNumber = 1 SndNumber = 3 Letter = 'a' Password = ""abcde"" }",1-3 a: abcde,OK
1,"Some({ FstNumber = 1 SndNumber = 3 Letter = 'b' Password = ""cdefg"" })","{ FstNumber = 1 SndNumber = 3 Letter = 'b' Password = ""cdefg"" }",1-3 b: cdefg,OK
2,"Some({ FstNumber = 2 SndNumber = 9 Letter = 'c' Password = ""ccccccccc"" })","{ FstNumber = 2 SndNumber = 9 Letter = 'c' Password = ""ccccccccc"" }",2-9 c: ccccccccc,OK


Let's solve the puzzle using the actual input.

In [1]:
#!fsharp
let puzzleInputUrl = "https://raw.githubusercontent.com/aaronmu/AoC/main/puzzle-input/day2.input"

In [1]:
#!fsharp
#r "nuget: FSharp.Data"

open FSharp.Data
open System
    
let parseInput (str: string) =
    str.Split "\n" |> Seq.choose parseLine

puzzleInputUrl
|> Http.RequestString
|> parseInput
|> Seq.filter policy
|> Seq.length

569

## Part 2

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

In [1]:
#!fsharp
module Seq =
    // item1 helps with 1 based indexes.
    let item1 index = Seq.item (index - 1)

let policy2 ({ FstNumber = fstNumber; SndNumber = sndNumber; Letter = letter; Password = password }: Input): bool =
    let letterAtFirstPos = Seq.item1 fstNumber password
    let letterAtSecondPos = Seq.item1 sndNumber password
    (letterAtFirstPos = letter) <> (letterAtSecondPos = letter)

    // Test the policy using the example provided in the puzzle.
[
    {| Input = { FstNumber = 1; SndNumber = 3; Letter = 'a'; Password = "abcde" }; ExpectedValid = true |}
    {| Input = { FstNumber = 1; SndNumber = 3; Letter = 'b'; Password = "cdefg" }; ExpectedValid = false |}
    {| Input = { FstNumber = 2; SndNumber = 9; Letter = 'c'; Password = "ccccccccc" }; ExpectedValid = false |}
] |> Seq.map (fun test -> 
    let actual = policy2 test.Input
    let result = if actual = test.ExpectedValid then "OK" else "NOK"
    {| test with ActualValid = policy test.Input; Result = result |})

index,ActualValid,ExpectedValid,Input,Result
0,True,True,"{ FstNumber = 1 SndNumber = 3 Letter = 'a' Password = ""abcde"" }",OK
1,False,False,"{ FstNumber = 1 SndNumber = 3 Letter = 'b' Password = ""cdefg"" }",OK
2,True,False,"{ FstNumber = 2 SndNumber = 9 Letter = 'c' Password = ""ccccccccc"" }",OK


We can reuse most of the part 1 functions to compose our part 2 solution.

In [1]:
#!fsharp
puzzleInputUrl
|> Http.RequestString
|> parseInput
|> Seq.filter policy2 // note that we use the policy2 function here instead of the original policy
|> Seq.length

346

## FParsec

I've been eager to learn FParsec for a long time but haven't really gotten to it. Advent of Code seems like a great opportunity to practice.

In [1]:
#!fsharp
#r "nuget: FParsec"

open FParsec

let fparse (x: string) =
    let ws = spaces
    let str x = pstring x .>> ws
    let number = pint32 .>> ws
    let matchLetter = letter .>> ws

    let parser = 
        many (
            ws >>.                      // skip whitespace
            number .>>                  // collect a number
            str "-" .>>.                // we expect the str "-"
            number .>>.                 // collect a number
            matchLetter .>>             // collect a single letter
            str ":" .>>.                // we expect the str ":"
            manySatisfy isLetter .>>    // collect a sequence of letters
            ws                          // skip whitespace
        )


    // After running the parser, we turn collected value into a CheckPassword.
    match run parser x with
    | Success (results, _, _) ->
        seq {
            for (((min, max), theLetter), password) in results do
                Input.create min max theLetter password
        }
    | Failure (errorMsg, _, _) -> failwith errorMsg

// Test the FParsec parser using the example provided in the puzzle.
// Note that this is the same "testsuite" used for testing the regular expression parser.
[
    {| Input = "1-3 a: abcde"; ExpectedOutput = [ { FstNumber = 1; SndNumber = 3; Letter = 'a'; Password = "abcde" } ] |}
    {| Input = "1-3 b: cdefg"; ExpectedOutput = [ { FstNumber = 1; SndNumber = 3; Letter = 'b'; Password = "cdefg" } ] |}
    {| Input = "2-9 c: ccccccccc"; ExpectedOutput = [ { FstNumber = 2; SndNumber = 9; Letter = 'c'; Password = "ccccccccc" } ] |}
] |> Seq.map (fun test -> 
    let actualOutput = fparse test.Input
    let result = if (List.ofSeq actualOutput) = test.ExpectedOutput then "OK" else "NOK"
    {| test with ActualOutput = actualOutput; Result = result |})

index,ActualOutput,ExpectedOutput,Input,Result
0,"[ { FstNumber = 1 SndNumber = 3 Letter = 'a' Password = ""abcde"" } ]","[ { FstNumber = 1 SndNumber = 3 Letter = 'a' Password = ""abcde"" } ]",1-3 a: abcde,OK
1,"[ { FstNumber = 1 SndNumber = 3 Letter = 'b' Password = ""cdefg"" } ]","[ { FstNumber = 1 SndNumber = 3 Letter = 'b' Password = ""cdefg"" } ]",1-3 b: cdefg,OK
2,"[ { FstNumber = 2 SndNumber = 9 Letter = 'c' Password = ""ccccccccc"" } ]","[ { FstNumber = 2 SndNumber = 9 Letter = 'c' Password = ""ccccccccc"" } ]",2-9 c: ccccccccc,OK


The regex parser was only capable of parsing individual lines but the FParsec parser easily handles multiple lines. It even deals with whitespace!

In [1]:
#!fsharp
let multiLineInput = """


9-12 q: qqqxhnhdmqqqqjz

12-16z: zzzzzznwlzzjzdzf        4-7s:sssgssw



13-14 p: pppqzpppppppfpppp 2-9 m: jknmmmmmmdmmmrm



"""

let expectedOutput = [
    { FstNumber = 9; SndNumber = 12; Letter = 'q'; Password = "qqqxhnhdmqqqqjz" }
    { FstNumber = 12; SndNumber = 16; Letter = 'z'; Password = "zzzzzznwlzzjzdzf" }
    { FstNumber = 4; SndNumber = 7; Letter = 's'; Password = "sssgssw" }
    { FstNumber = 13; SndNumber = 14; Letter = 'p'; Password = "pppqzpppppppfpppp" }
    { FstNumber = 2; SndNumber = 9; Letter = 'm'; Password = "jknmmmmmmdmmmrm" }
]

Seq.toList (fparse multiLineInput) = expectedOutput

True

Note that we can easily compose our new parser in our solution. That's the power of functions right here.

In [1]:
#!fsharp
puzzleInputUrl
|> Http.RequestString
|> fparse
|> Seq.filter policy2 // note that we use the policy2 function here instead of the original policy
|> Seq.length

346

So now I'm wondering which is faster. The regex parser or FParsec?

In [1]:
#!fsharp
let input = 
    Http.RequestString puzzleInputUrl
    |> List.replicate 100
    |> String.concat "\n"

// Utility function that allows us to time a function.
let time fn =
    let timer = new System.Diagnostics.Stopwatch()
    timer.Start()
    let _returnValue = fn ()

    {| 
        ElapsedMilliseconds = timer.ElapsedMilliseconds
        ElapsedTicks = timer.ElapsedTicks
    |}

// I use Seq.length in the measurements because the regex parser is lazy.
{|
    RegexParse = time (fun _unit -> input |> parseInput |> Seq.length)
    FParsecParse = time (fun _unit -> input |> fparse |> Seq.length)
|}



FParsecParse,RegexParse
{ ElapsedMilliseconds = 45L ElapsedTicks = 45026050L },{ ElapsedMilliseconds = 167L ElapsedTicks = 167639251L }


This result really surprised me. I still don't believe it. It looks like the FParsec parser outperforms the regular expression with ease. That's amazing.